In [8]:
import re
import jieba
import pandas as pd

# 0. gensim实践：

1. 读取预处理好的数据
2. 训练
3. 完事

# 1. 数据集路径

In [33]:
merger_data_path = 'data/merged_train_test_seg_data.csv'

# 2. 载入数据

In [11]:
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
print('train data size {},test data size {}'.format(len(train_df),len(test_df)))
train_df.head()

train data size 82871,test data size 20000


,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1 进口,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位 ...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场 价钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


# 3. 模型创建

Gensim中 Word2Vec 模型的期望输入是进过分词的句子列表，即是某个二维数组。这里我们暂时使用 Python 内置的数组，不过其在输入数据集较大的情况下会占用大量的 RAM。Gensim 本身只是要求能够迭代的有序句子列表，因此在工程实践中我们可以使用自定义的生成器，只在内存中保存单条语句。

## Word2Vec 参数
+ min_count

在不同大小的语料集中，我们对于基准词频的需求也是不一样的。譬如在较大的语料集中，我们希望忽略那些只出现过一两次的单词，这里我们就可以通过设置min_count参数进行控制。一般而言，合理的参数值会设置在0~100之间。

+ size

size参数主要是用来设置神经网络的层数，Word2Vec 中的默认值是设置为100层。更大的层次设置意味着更多的输入数据，不过也能提升整体的准确度，合理的设置范围为 10~数百。

+ workers

workers参数用于设置并发训练时候的线程数，不过仅当Cython安装的情况下才会起作用：

In [12]:
# 引入 word2vec
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
import gensim

# 引入日志配置
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 构建训练

In [21]:
model = word2vec.Word2Vec(LineSentence(merger_data_path), workers=8,min_count=0,size=200)

2019-11-17 16:39:41,459 : INFO : collecting all words and their counts
2019-11-17 16:39:41,460 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-17 16:39:41,677 : INFO : PROGRESS: at sentence #10000, processed 928198 words, keeping 41567 word types
2019-11-17 16:39:41,892 : INFO : PROGRESS: at sentence #20000, processed 1870800 words, keeping 62216 word types
2019-11-17 16:39:42,104 : INFO : PROGRESS: at sentence #30000, processed 2802551 words, keeping 77593 word types
2019-11-17 16:39:42,317 : INFO : PROGRESS: at sentence #40000, processed 3706533 words, keeping 90875 word types
2019-11-17 16:39:42,541 : INFO : PROGRESS: at sentence #50000, processed 4670345 words, keeping 102969 word types
2019-11-17 16:39:42,782 : INFO : PROGRESS: at sentence #60000, processed 5694540 words, keeping 115297 word types
2019-11-17 16:39:43,019 : INFO : PROGRESS: at sentence #70000, processed 6741207 words, keeping 127165 word types
2019-11-17 16:39:43,233 : INFO : PROG

2019-11-17 16:40:08,159 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-11-17 16:40:08,160 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-11-17 16:40:08,160 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-11-17 16:40:08,161 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-17 16:40:08,167 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-17 16:40:08,170 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-17 16:40:08,170 : INFO : EPOCH - 5 : training on 9610735 raw words (8683775 effective words) took 4.5s, 1943460 effective words/s
2019-11-17 16:40:08,171 : INFO : training on a 48053675 raw words (43414637 effective words) took 22.3s, 1943881 effective words/s


# 查找最近的词

In [15]:
model.wv.most_similar(['奇瑞'],topn=10)

[('海马', 0.8893340826034546),
 ('东南', 0.876111626625061),
 ('二代', 0.8668296337127686),
 ('铃木', 0.8638443946838379),
 ('名爵', 0.8610075116157532),
 ('猎豹', 0.856071412563324),
 ('东风风行', 0.8480671048164368),
 ('江淮', 0.846108615398407),
 ('瑞虎5', 0.8452643752098083),
 ('东风', 0.8447585701942444)]

# 保存模型

In [34]:
save_model_path='data/wv/word2vec.model'

In [27]:
model.save(save_model_path)

2019-11-17 16:42:40,274 : INFO : saving Word2Vec object under data/wv/word2vec.model, separately None
2019-11-17 16:42:40,277 : INFO : storing np array 'vectors' to data/wv/word2vec.model.wv.vectors.npy
2019-11-17 16:42:40,404 : INFO : not storing attribute vectors_norm
2019-11-17 16:42:40,408 : INFO : storing np array 'syn1neg' to data/wv/word2vec.model.trainables.syn1neg.npy
2019-11-17 16:42:40,537 : INFO : not storing attribute cum_table
2019-11-17 16:42:40,760 : INFO : saved data/wv/word2vec.model


# 载入模型

In [29]:
model = word2vec.Word2Vec.load(save_model_path)

2019-11-17 16:43:21,044 : INFO : loading Word2Vec object from data/wv/word2vec.model
2019-11-17 16:43:21,262 : INFO : loading wv recursively from data/wv/word2vec.model.wv.* with mmap=None
2019-11-17 16:43:21,262 : INFO : loading vectors from data/wv/word2vec.model.wv.vectors.npy with mmap=None
2019-11-17 16:43:21,292 : INFO : setting ignored attribute vectors_norm to None
2019-11-17 16:43:21,293 : INFO : loading vocabulary recursively from data/wv/word2vec.model.vocabulary.* with mmap=None
2019-11-17 16:43:21,293 : INFO : loading trainables recursively from data/wv/word2vec.model.trainables.* with mmap=None
2019-11-17 16:43:21,293 : INFO : loading syn1neg from data/wv/word2vec.model.trainables.syn1neg.npy with mmap=None
2019-11-17 16:43:21,323 : INFO : setting ignored attribute cum_table to None
2019-11-17 16:43:21,324 : INFO : loaded data/wv/word2vec.model


In [30]:
model.wv.most_similar(['奇瑞'],topn=10)

2019-11-17 16:43:28,803 : INFO : precomputing L2-norms of word weight vectors


[('东南', 0.8724187016487122),
 ('瑞虎', 0.8533821105957031),
 ('二代', 0.8516823053359985),
 ('名爵', 0.847603440284729),
 ('江淮', 0.8443121910095215),
 ('海马', 0.8403477072715759),
 ('瑞虎5', 0.8393649458885193),
 ('东风风行', 0.8331546783447266),
 ('东风', 0.8330514430999756),
 ('铃木', 0.8277937769889832)]

# 参考

1. https://radimrehurek.com/gensim/models/word2vec.html 